# Assignmnet 2
## Arose Niazi (FA18-BSE-010)

### Importing Libraries

In [1]:
import folium
import pandas as pd
import numpy as np
import requests
import json

### Presetup

In [2]:
pakistan_loc = [30.3753,69.3451]

### Creating World Map

In [3]:
pakistan  = folium.Map(
    location=pakistan_loc,
    zoom_start=5
)
pakistan

### Working on Maps / Adding Multiple Maps
#### Adding layers to the map

In [4]:
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    folium.TileLayer(tile).add_to(pakistan)

#### Making more maps for multiple data

In [5]:
cases_map = folium.Map(
    location=pakistan_loc,
    tiles='cartodbpositron',
    zoom_start=5
)

for tile in tiles:
    folium.TileLayer(tile).add_to(cases_map)

deaths_map = folium.Map(
    location=pakistan_loc,
    tiles='cartodbpositron',
    zoom_start=5
)

for tile in tiles:
    folium.TileLayer(tile).add_to(deaths_map)

### Data Collection
#### Scrapping Website for locations data

In [6]:
info = pd.read_html('https://mm-ftw.net/pov.html')
provinces = pd.DataFrame(info[0])
provinces

,IDs,Province,Latitude,Longitude
0,JK,Azad Jammu and Kashmir,33.9259,73.7810
1,BL,Balochistan,28.4907,65.0958
2,FT,F.A.T.A,32.6675,69.8597
3,IS,Islamabad Capital Territory,33.7205,73.0405
4,KP,Khyber Pakhtunkhwa,34.9526,72.3311
5,GB,Gilgit-Baltistan,35.8026,74.9832
6,PU,Punjab,31.1471,72.7097
7,SH,Sindh,25.8943,68.5247
8,OJK,Occupied Jammu and Kashmir,32.7266,74.8570


#### Scrapping Website for COVID-19 data

In [7]:
info = pd.read_html("https://news.google.com/covid19/map?hl=en-PK&mid=%2Fm%2F05sb1&gl=PK&ceid=PK%3Aen", match="Cases")
covid19 = pd.DataFrame(info[0])
covid19.head()

,Location,Total cases,New cases (1 day*),New cases (last 60 days),Cases per 1 million people,Deaths
0,Worldwide,92313199,566186,NaN,11872,1977893
1,Pakistan,511921,3097,NaN,2336,10818
2,Sindh,230718,1769,NaN,4818,3744
3,Punjab,147292,767,NaN,No data,4348
4,Khyber Pakhtunkhwa,62377,359,NaN,1756,1752


#### Data Cleaning

In [8]:
#Convert to DataFrame
covid19 = pd.DataFrame(info[0])
covid19 = covid19.iloc[2:9]
covid19 = covid19.drop(["New cases (1 day*)", "New cases (last 60 days)", "Cases per 1 million people"], axis=1)
covid19 = covid19.rename(columns={'Location': 'Province'})
covid19['Total cases'] = covid19['Total cases'].astype('int')
covid19['Deaths'] = covid19['Deaths'].astype('int')
covid19

,Province,Total cases,Deaths
2,Sindh,230718,3744
3,Punjab,147292,4348
4,Khyber Pakhtunkhwa,62377,1752
5,Islamabad Capital Territory,39624,449
6,Balochistan,18488,188
7,Azad Jammu and Kashmir,8540,236
8,Gilgit-Baltistan,4882,101


#### Joining data / Clearning it / Calculating Data

In [9]:
final_data = pd.merge(provinces, covid19, how ='outer', on ='Province')
fdata=final_data.sum(axis = 0, skipna = True) 
final_data['Deaths'] = final_data['Deaths'].fillna(0)
final_data['Total cases'] = final_data['Total cases'].fillna(0)
final_data['Death Ratio'] = (final_data['Deaths'] / final_data['Total cases']) * 100
final_data['Cases Ratio'] = (final_data['Total cases'] / fdata['Total cases']) * 100
final_data['Death Ratio'] = final_data['Death Ratio'].fillna(0)
final_data

,IDs,Province,Latitude,Longitude,Total cases,Deaths,Death Ratio,Cases Ratio
0,JK,Azad Jammu and Kashmir,33.9259,73.7810,8540.0,236.0,2.763466,1.668226
1,BL,Balochistan,28.4907,65.0958,18488.0,188.0,1.016876,3.611495
2,FT,F.A.T.A,32.6675,69.8597,0.0,0.0,0.000000,0.000000
3,IS,Islamabad Capital Territory,33.7205,73.0405,39624.0,449.0,1.133152,7.740257
4,KP,Khyber Pakhtunkhwa,34.9526,72.3311,62377.0,1752.0,2.808728,12.184888
5,GB,Gilgit-Baltistan,35.8026,74.9832,4882.0,101.0,2.068824,0.953663
6,PU,Punjab,31.1471,72.7097,147292.0,4348.0,2.951959,28.772408
7,SH,Sindh,25.8943,68.5247,230718.0,3744.0,1.622760,45.069063
8,OJK,Occupied Jammu and Kashmir,32.7266,74.8570,0.0,0.0,0.000000,0.000000


### Map Data
#### Loading map data from GeoJson files

In [10]:
geoUrl = 'https://raw.githubusercontent.com/Arose-Niazi/Pakistan-latest-map-GeoJSON/main/Pakistan.json'
data_geojson_dict = json.loads(requests.get(geoUrl).text)

#### Working on first map; which shows the details

In [11]:
folium.GeoJson(
    geoUrl,
    name='geojson'
).add_to(pakistan)

layer_geom = folium.FeatureGroup(name='layer',control=False)



Adding markers to the map

In [14]:
for province,lat,long,total_cases,Death,dratio,cratio in zip(list(final_data['Province']),list(final_data['Latitude']),list(final_data['Longitude']),list(final_data['Total cases']),list(final_data['Deaths']),list(final_data['Death Ratio']),list(final_data['Cases Ratio'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(pakistan)
    popup_html =str('<strong><b>Province: '+province+'</strong> <br>' +
                    '<strong><b>Total Cases :'+str(total_cases)+'</striong><br>' +
                    '<strong><b>Deaths :'+str(Death)+'</striong><br>' +
                    '<strong><b>Death Ratio :'+str("%0.2f%%" % dratio)+'</striong><br>' +
                    '<strong><b>Cases Ratio :'+str("%0.2f%%" % cratio)+'</striong>')
    folium.Marker(location = [lat,long],
              popup = folium.Popup(popup_html, max_width=300), icon = folium.Icon(color='red', icon='map-pin',  prefix='fa') ).add_to(pakistan)

Adding popus for simple clicks

In [15]:
provinces = []

for i in range(len(data_geojson_dict["features"])):
    temp_geojson = {
        "features":[data_geojson_dict["features"][i]],
        "type":"FeatureCollection"
    } 
    temp_geojson_layer = folium.GeoJson(temp_geojson,
        highlight_function=lambda x: {'weight':3, 'color': 'red'},
        control=False,
        style_function=lambda feature: {'color': "green",'weight': 3})
    popupstr = '<b>'+ final_data['Province'][i] +'</b><br>' + '<b>Total Cases :'+str(final_data['Total cases'][i])+'</b><br>' +'<b>Deaths :'+str(final_data['Deaths'][i])+'</b><br>'
        
    folium.Popup(popupstr, max_width = 500).add_to(temp_geojson_layer)
    temp_geojson_layer.add_to(layer_geom)

layer_geom.add_to(pakistan)
folium.LayerControl(autoZIndex=False, collapsed=False).add_to(pakistan)

#### Working on second map; which shows cases ratio

Creating a tooltip to add for cases

In [16]:
tooltip_text = []
for idx in range(len(final_data)):
 tooltip_text.append(final_data["Province"][idx] +"</BR>Cases:"+ str(final_data["Total cases"][idx])+"</BR>Ratio:"+ str("%0.2f%%" % (final_data["Cases Ratio"][idx])))

Append a tooltip column with customised text

In [17]:
for idx in range(len(tooltip_text)):
    data_geojson_dict['features'][idx]['properties']['cases_tt'] = tooltip_text[idx]

Adding data to the map

In [18]:
choropleth = folium.Choropleth(
    geo_data=data_geojson_dict,
    name='choropleth',
    data=final_data,
    columns=['IDs', 'Cases Ratio'],
    key_on='feature.id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cases Ratio',
    highlight = True
).add_to(cases_map)

folium.LayerControl().add_to(cases_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['cases_tt'], labels=False)
)

#### Working on third map; which shows deaths ratio

Creating a tooltip to add for deaths and appeding the tooltip column with customised text

In [19]:
tooltip_text = []
for idx in range(len(final_data)):
 tooltip_text.append(final_data["Province"][idx] +"</BR>Cases:"+ str(final_data["Total cases"][idx])+"</BR>Deaths:"+ str(final_data["Deaths"][idx])+"</BR>Ratio:"+ str("%0.2f%%" % (final_data["Death Ratio"][idx])))
for idx in range(len(tooltip_text)):
    data_geojson_dict['features'][idx]['properties']['deaths_tt'] = tooltip_text[idx]

Adding data to the map

In [20]:
choropleth = folium.Choropleth(
    geo_data=data_geojson_dict,
    name='choropleth',
    data=final_data,
    columns=['IDs', 'Death Ratio'],
    key_on='feature.id',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cases to Death Ratio',
    highlight = True
).add_to(deaths_map)

folium.LayerControl().add_to(deaths_map)

choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['deaths_tt'], labels=False)
)

### Saving to file

In [21]:
pakistan.save('Covid_Statistics.html')
cases_map.save('Covid_Cases.html')
deaths_map.save('Covid_Deaths.html')

### Displayng the maps

#### Pakistan COVID Map
Click the province to reveal stats

In [22]:
pakistan

#### Pakistan Cases map
Hover the province to reveal stats

In [23]:
cases_map

#### Pakistan Deaths map
Hover the province to reveal stats

In [24]:
deaths_map